In [2]:
# import libraries
import pandas as pd
import numpy as np
# datetime
from datetime import datetime, timedelta
# functools for reduce
from functools import reduce

### Preparing data

In [3]:
# import calls df
calls = pd.read_csv('../data/raw/Calls.csv')
# drop duplicates
# calls = calls.drop_duplicates()
# display head
display(calls.head())
# display calls info
display(calls.info())
# calls shape of data
print('calls shape: ', calls.shape)

,Date,CallTime,NumberOfCalls,Вид учета времени,InOut,id
0,"2021-08-16 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
1,"2021-09-21 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
2,"2021-01-11 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
3,"2021-01-18 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945
4,"2021-01-27 00:00:00,000","0,00027777777777777778",1,Будни,ToUser,ОРГ1-01945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407867 entries, 0 to 407866
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date               407867 non-null  object
 1   CallTime           407867 non-null  object
 2   NumberOfCalls      407867 non-null  int64 
 3   Вид учета времени  392655 non-null  object
 4   InOut              407867 non-null  object
 5   id                 407867 non-null  object
dtypes: int64(1), object(5)
memory usage: 18.7+ MB


None

calls shape:  (407867, 6)


In [4]:
# date to datetime
calls['date_calls'] = pd.to_datetime(calls['Date'].apply(lambda x: x[:10]), format='%Y-%m-%d')
# drop date column
calls.drop('Date', axis=1, inplace=True)

In [5]:
# information about period of data
max_date = calls['date_calls'].max()
print('Data starts at: ', calls['date_calls'].min().date())
print('Data ends at : ', calls['date_calls'].max().date())
print('Total days presented: ', calls['date_calls'].nunique())
print('Period of data (days): ', (calls['date_calls'].max() - calls['date_calls'].min()).days + 1)

Data starts at:  2021-01-01
Data ends at :  2021-12-30
Total days presented:  364
Period of data (days):  364


In [6]:
# CallTime convert to float
calls['CallTime'] = calls['CallTime'].apply(lambda x: float(x.replace(',','.')))

In [7]:
# select all dates and weekend marks
dates = calls[['date_calls', 'Вид учета времени']].dropna().sort_values('date_calls').reset_index(drop=True)
# weekend marks for fill na
weekeend_days_map = dates.groupby('date_calls')['Вид учета времени'].agg(pd.Series.mode).to_dict()

In [8]:
# fill na with modes type of data
calls['Вид учета времени'] = np.where(calls['Вид учета времени'].isnull(), 
                                      calls['date_calls'].map(weekeend_days_map), calls['Вид учета времени'])

In [9]:
# agg function to calculate work period by first and last calls
def work_period_by_calls(dates):
    return (dates.max() - dates.min()).days

In [10]:
# agg function to find days since last call
def days_since_last_call(dates, max_date=max_date):
    return (max_date - dates.max()).days

In [11]:
def call_time_features(calls):
    # mean calltime less than 10 seconds on day goes to blank ( 10 seconds / 3600 seconds)
    calls['less_10_secs_call'] = np.where(
        calls['CallTime']/calls['NumberOfCalls'] < 0.0028, 1, 0)
    # mean calltime more than 20 seconds
    calls['between_10_and_60_secs_call'] = np.where(
        (calls['CallTime']/calls['NumberOfCalls']).between(0.0028, 0.017), 1, 0)
    # mean calltime more than 1 minutes
    calls['more_60_secs_call'] = np.where(
        calls['CallTime']/calls['NumberOfCalls'] > 0.017, 1, 0)
    # mean calltime more than 3 minutes
    calls['more_180_secs_call'] = np.where(
        calls['CallTime']/calls['NumberOfCalls'] > 0.05, 1, 0)
    # mean calltime more than 5 minutes
    calls['more_300_secs_call'] = np.where(
        calls['CallTime']/calls['NumberOfCalls'] > 0.084, 1, 0)
    # mean calltime more than 10 minutes
    calls['more_600_secs_call'] = np.where(
        calls['CallTime']/calls['NumberOfCalls'] > 0.167, 1, 0)
    
    # holiday-workdays marks
    calls['weekend_work_days_calls'] = np.where(calls['Вид учета времени'] == 'Выходные дни', 1 , 0)
    calls['workday_work_days_calls'] = np.where(calls['Вид учета времени'] == 'Будни', 1, 0)
    
    
    return calls

### aggregations

In [12]:
# types of call
print('Types of call: ', calls['InOut'].unique())

Types of call:  ['ToUser' 'FromUser']


In [13]:
# divide df to ToUser and FromUser calls for aggregation
calls_to_user = calls[calls['InOut'] == 'ToUser']
calls_from_user = calls[calls['InOut'] == 'FromUser']

In [14]:
# df for last 150 days
calls_last_150 = calls[calls['date_calls'] > max_date - timedelta(days=150) ]
# df for last 100 days
calls_last_100 = calls[calls['date_calls'] > max_date - timedelta(days=100) ]
#df for last 50 days
calls_last_50 = calls[calls['date_calls'] > max_date - timedelta(days=50)]

In [15]:
# df for last 150 days calls to user and from user
calls_last_150_to_user = calls_last_150[calls_last_150['InOut'] == 'ToUser']
calls_last_150_from_user = calls_last_150[calls_last_150['InOut'] == 'FromUser']
# df for last 100 days calls to user and from user
calls_last_100_to_user = calls_last_100[calls_last_100['InOut'] == 'ToUser']
calls_last_100_from_user = calls_last_100[calls_last_100['InOut'] == 'FromUser']
# df for last 50 days calls to user and from user
calls_last_50_to_user = calls_last_50[calls_last_50['InOut'] == 'ToUser']
calls_last_50_from_user = calls_last_50[calls_last_50['InOut'] == 'FromUser']


In [16]:
calls_dfs_for_aggregation = [calls, calls_to_user, calls_from_user, 
                             calls_last_150, calls_last_100, calls_last_50,
                             calls_last_150_to_user, calls_last_150_from_user, 
                             calls_last_100_to_user, calls_last_100_from_user,
                             calls_last_50_to_user, calls_last_50_from_user]

In [17]:
calls = calls.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_to_user = calls_to_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_from_user = calls_from_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_150 = calls_last_150.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_100 = calls_last_100.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_50 = calls_last_50.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_150_to_user = calls_last_150_to_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_150_from_user = calls_last_150_from_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_100_to_user = calls_last_100_to_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_100_from_user = calls_last_100_from_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_50_to_user = calls_last_50_to_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()
calls_last_50_from_user = calls_last_50_from_user.groupby(['Вид учета времени', 'InOut', 'id', 'date_calls'], as_index=False).sum()


In [18]:
calls_dfs_for_aggregation = [calls, calls_to_user, calls_from_user, 
                             calls_last_150, calls_last_100, calls_last_50,
                             calls_last_150_to_user, calls_last_150_from_user, 
                             calls_last_100_to_user, calls_last_100_from_user,
                             calls_last_50_to_user, calls_last_50_from_user]

In [19]:
# add features
for df in calls_dfs_for_aggregation:
    df = call_time_features(df)

In [20]:
# create features about last call since IN and FROM calls
# aggregation
since_last_calls_to_user = calls[calls['InOut'] == 'FromUser'].groupby('id', as_index=False).agg({
                    'date_calls': days_since_last_call})

since_last_calls_from_user = calls[calls['InOut'] == 'ToUser'].groupby('id', as_index=False).agg({
                    'date_calls': days_since_last_call})
        
# merge dfs
last_calls = pd.merge(since_last_calls_to_user,
                      since_last_calls_from_user, how='outer', on='id')
# rename columns
last_calls.columns = ['id', 'days_since_last_call_from_user', 'days_since_last_call_to_user']

#fill N/A's with 364 (that means - never calls)
last_calls.fillna(364, inplace=True)

In [21]:
def aggregation_calls(calls):
    calls = calls.groupby('id', as_index=False).agg({
        'date_calls': ['nunique', work_period_by_calls, days_since_last_call],
        'CallTime': ['mean', 'sum', 'count', 'std', 'min', 'max'],
        'NumberOfCalls': ['mean', 'sum', 'count', 'std', 'min', 'max'],
        'less_10_secs_call': ['mean', 'sum'],
        'between_10_and_60_secs_call': ['mean', 'sum'],
        'more_60_secs_call': ['mean', 'sum'],
        'more_180_secs_call': ['mean', 'sum'],
        'more_300_secs_call': ['mean', 'sum'],
        'more_600_secs_call': ['mean', 'sum'],
        'weekend_work_days_calls': ['mean', 'sum'],
        'workday_work_days_calls': ['mean', 'sum']
    })
    calls.columns = calls.columns.map('_'.join).str.strip('_')
    calls['true_calltime_mean'] = calls['CallTime_sum'] / calls['NumberOfCalls_sum']
    return calls

In [22]:
calls = aggregation_calls(calls)
calls_to_user = aggregation_calls(calls_to_user)
calls_from_user = aggregation_calls(calls_from_user)
calls_last_150 = aggregation_calls(calls_last_150)
calls_last_100 = aggregation_calls(calls_last_100)
calls_last_50 = aggregation_calls(calls_last_50)
calls_last_150_to_user = aggregation_calls(calls_last_150_to_user)
calls_last_150_from_user = aggregation_calls(calls_last_150_from_user)
calls_last_100_to_user = aggregation_calls(calls_last_100_to_user)
calls_last_100_from_user = aggregation_calls(calls_last_100_from_user)
calls_last_50_to_user = aggregation_calls(calls_last_50_to_user)
calls_last_50_from_user = aggregation_calls(calls_last_50_from_user)

In [23]:
# add suffixes to dfs columns 
calls.columns = calls.columns.map(lambda x: x+'_total' if x != 'id' else x)
calls_to_user.columns = calls_to_user.columns.map(
    lambda x: x+'_to_user' if x != 'id' else x)
calls_from_user.columns = calls_from_user.columns.map(
    lambda x: x+'_from_user' if x != 'id' else x)
calls_last_150.columns = calls_last_150.columns.map(
    lambda x: x+'_last_150' if x != 'id' else x)
calls_last_100.columns = calls_last_100.columns.map(
    lambda x: x+'_last_100' if x != 'id' else x)
calls_last_50.columns = calls_last_50.columns.map(
    lambda x: x+'_last_50' if x != 'id' else x)
calls_last_150_to_user.columns = calls_last_150_to_user.columns.map(
    lambda x: x+'_last_150_to_user' if x != 'id' else x)
calls_last_150_from_user.columns = calls_last_150_from_user.columns.map(
    lambda x: x+'_last_150_from_user' if x != 'id' else x)
calls_last_100_to_user.columns = calls_last_100_to_user.columns.map(
    lambda x: x+'_last_100_to_user' if x != 'id' else x)
calls_last_100_from_user.columns = calls_last_100_from_user.columns.map(
    lambda x: x+'_last_100_from_user' if x != 'id' else x)
calls_last_50_to_user.columns = calls_last_50_to_user.columns.map(
    lambda x: x+'_last_50_to_user' if x != 'id' else x)
calls_last_50_from_user.columns = calls_last_50_from_user.columns.map(
    lambda x: x+'_last_50_from_user' if x != 'id' else x)

In [24]:
calls_dfs_for_merge = [calls, calls_to_user, calls_from_user, 
                             calls_last_150, calls_last_100, calls_last_50,
                             calls_last_150_to_user, calls_last_150_from_user, 
                             calls_last_100_to_user, calls_last_100_from_user,
                             calls_last_50_to_user, calls_last_50_from_user, since_last_calls_to_user, since_last_calls_from_user]

grouped_data = reduce(lambda left, right: pd.merge(left, right, on=['id'],
                                            how='left'), calls_dfs_for_merge)

In [25]:
# fill na values with 0
grouped_data.fillna(0, inplace=True)

In [26]:
# to-from calls ratio
grouped_data['to_from_ratio_total'] = grouped_data['NumberOfCalls_sum_to_user'] / \
    (grouped_data['NumberOfCalls_sum_from_user'] + 1)
# to-from calls ratio last 150
grouped_data['to_from_last_150_ratio_total'] = grouped_data['NumberOfCalls_sum_last_150_to_user'] / \
    (grouped_data['NumberOfCalls_sum_last_150_from_user'] + 1)
# to-from calls ratio last 100
grouped_data['to_from_last_100_ratio_total'] = grouped_data['NumberOfCalls_sum_last_100_to_user'] / \
    (grouped_data['NumberOfCalls_sum_last_100_from_user'] + 1)
# to-from calls ratio last 50
grouped_data['to_from_last_50_ratio_total'] = grouped_data['NumberOfCalls_sum_last_50_to_user'] / \
    (grouped_data['NumberOfCalls_sum_last_50_from_user'] + 1)

In [27]:
grouped_data['exists_in_calls'] = 1

In [28]:
grouped_data.to_csv('../data/prepared/grouped_calls.csv', index=False)

In [29]:
grouped_data

,id,date_calls_nunique_total,date_calls_work_period_by_calls_total,date_calls_days_since_last_call_total,CallTime_mean_total,CallTime_sum_total,CallTime_count_total,CallTime_std_total,CallTime_min_total,CallTime_max_total,...,workday_work_days_calls_mean_last_50_from_user,workday_work_days_calls_sum_last_50_from_user,true_calltime_mean_last_50_from_user,date_calls_x,date_calls_y,to_from_ratio_total,to_from_last_150_ratio_total,to_from_last_100_ratio_total,to_from_last_50_ratio_total,exists_in_calls
0,ОРГ1-00004,199,352,0,0.409560,162.595278,397,0.296862,0.009167,1.832500,...,1.000000,35.0,0.028680,0.0,0.0,0.610020,0.606909,0.595048,0.684211,1
1,ОРГ1-00005,214,359,0,0.662028,272.755556,412,0.865917,0.000278,5.683333,...,0.942857,33.0,0.061033,0.0,0.0,0.664449,0.673699,0.715789,0.733119,1
2,ОРГ1-00028,215,353,0,0.151212,59.880000,396,0.271606,0.000278,2.938889,...,1.000000,33.0,0.034149,0.0,0.0,1.065411,1.110345,1.016949,0.865801,1
3,ОРГ1-00030,202,353,0,0.162173,62.760833,387,0.172911,0.003056,1.675833,...,1.000000,32.0,0.024228,0.0,0.0,0.848615,0.974533,0.925558,1.022523,1
4,ОРГ1-00044,225,358,0,0.361930,153.458333,424,0.469940,0.002222,6.196944,...,0.972973,36.0,0.043885,0.0,0.0,0.961576,1.010582,1.031457,1.016807,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,ОРГ2-05629,5,74,198,1.225611,6.128056,5,0.631943,0.424722,1.943056,...,0.000000,0.0,0.000000,198.0,0.0,0.000000,0.000000,0.000000,0.000000,1
1067,ОРГ2-05862,3,113,238,0.105093,0.315278,3,0.178179,0.001389,0.310833,...,0.000000,0.0,0.000000,238.0,0.0,0.000000,0.000000,0.000000,0.000000,1
1068,ОРГ2-05932,9,79,265,0.663735,5.973611,9,0.443417,0.000556,1.280278,...,0.000000,0.0,0.000000,265.0,0.0,0.000000,0.000000,0.000000,0.000000,1
1069,ОРГ2-06006,1,0,209,0.220278,0.220278,1,0.000000,0.220278,0.220278,...,0.000000,0.0,0.000000,209.0,0.0,0.000000,0.000000,0.000000,0.000000,1
